# Análise Emocional com SenticNet

Notebook para execução passo a passo.

In [12]:
import logging
from collections import Counter
import pandas as pd
from senticnet_data import senticnet
import numpy as np
from typing import Optional, List

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

## Passo 1: Carregar os dados

In [13]:
df = pd.read_csv('pre-processed.csv')  # Ajuste o nome se necessário
logging.info(f"Dados carregados: {len(df)} linhas")
df.head()

2025-06-23 09:54:26 INFO: Dados carregados: 7200 linhas


,index,label,preprocessed_news
0,0,fake,katia abreu diz vai colocar expulsao moldura n...
1,1,fake,ray peita bolsonaro conservador fake entrevist...
2,2,fake,reinaldo azevedo desmascarado policia federal ...
3,3,fake,relatorio assustador bndes mostra dinheiro pub...
4,4,fake,radialista americano fala sobre pt vendem ilus...


## Passo 2: Definir funções de análise de sentimentos

In [ ]:
def calcular_media(valores: List[float]) -> float:
    return sum(valores) / len(valores) if valores else np.nan

def analisar_sentimentos(texto: str) -> dict[str, float | int]:
    if not isinstance(texto, str): texto = ""
    introspections, tempers, attitudes, sensitivities, emotions = [], [], [], [], []
    for palavra in texto.split():
        if palavra in senticnet:
            introspection, temper, attitude, sensitivity, emotion, *_ = senticnet[palavra]
            introspections.append(introspection)
            tempers.append(temper)
            attitudes.append(attitude)
            sensitivities.append(sensitivity)
            emotions.append(emotion)
    contagem = Counter(emotions)
    return {
        'media_attitude': calcular_media(attitudes),
        'media_sensitivity': calcular_media(sensitivities),
        'media_temper': calcular_media(tempers),
        'media_introspection': calcular_media(introspections),
        'total_palavras': len(attitudes),
        'contagem_raiva': contagem.get('#anger', 0),
        'contagem_tristeza': contagem.get('#grief', 0),
        'contagem_medo': contagem.get('#fear', 0),
        'contagem_alegria': contagem.get('#joy', 0),
        'contagem_surpresa': contagem.get('#surprise', 0),
    }

## Passo 3: Aplicar análise

In [15]:
resultados_df = df['preprocessed_news'].apply(analisar_sentimentos).apply(pd.Series)
resultados_df.head()

,media_attitude,media_sensitivity,media_temper,media_introspection,total_palavras,contagem_raiva,contagem_tristeza,contagem_medo,contagem_alegria,contagem_surpresa
0,-0.102771,0.069000,0.021943,-0.153571,35.0,0.0,10.0,1.0,0.0,0.0
1,-0.307909,0.040500,0.053705,-0.359432,44.0,0.0,21.0,1.0,0.0,0.0
2,0.016310,0.044119,0.008738,-0.000857,42.0,2.0,7.0,0.0,0.0,0.0
3,0.044173,0.121037,0.004235,0.142753,81.0,0.0,11.0,1.0,1.0,0.0
4,-0.218704,0.101074,-0.030852,-0.321667,27.0,1.0,14.0,0.0,2.0,0.0


## Passo 4: Concatenar e salvar resultados

In [16]:
df_final = pd.concat([df, resultados_df], axis=1)
output = 'analise_emocional_completa.csv'
df_final.to_csv(output, index=False)
print(f"Arquivo salvo em {output}")

Arquivo salvo em analise_emocional_completa.csv


## Passo 5: Visualizar resultados finais

In [ ]:
df_final.head()

,index,label,preprocessed_news,media_attitude,media_sensitivity,media_temper,media_introspection,total_palavras,contagem_raiva,contagem_tristeza,contagem_medo,contagem_alegria,contagem_surpresa
0,0,fake,katia abreu diz vai colocar expulsao moldura n...,-0.102771,0.069000,0.021943,-0.153571,35.0,0.0,10.0,1.0,0.0,0.0
1,1,fake,ray peita bolsonaro conservador fake entrevist...,-0.307909,0.040500,0.053705,-0.359432,44.0,0.0,21.0,1.0,0.0,0.0
2,2,fake,reinaldo azevedo desmascarado policia federal ...,0.016310,0.044119,0.008738,-0.000857,42.0,2.0,7.0,0.0,0.0,0.0
3,3,fake,relatorio assustador bndes mostra dinheiro pub...,0.044173,0.121037,0.004235,0.142753,81.0,0.0,11.0,1.0,1.0,0.0
4,4,fake,radialista americano fala sobre pt vendem ilus...,-0.218704,0.101074,-0.030852,-0.321667,27.0,1.0,14.0,0.0,2.0,0.0


: 